In [ ]:
from matplotlib import animation
import ray

from cleaner.cleaner_env import *

In [ ]:
config_name = "simple_3x3"
run_name = "second_run"
checkpoint_num = 1001

ray.shutdown()
ray.init()
config = load_config(config_name)
# evaluate(config, run_name, checkpoint_num, record=True)